# Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np

from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import data

In [ ]:
path = '/content/drive/MyDrive/Hackaton2023/train_dataset_Росатом/'
directory =  '/content/drive/MyDrive/Hackaton2023/train_dataset_Росатом/Anonimized/'

anonimized = pd.read_csv(path + 'anonimized.csv')

In [ ]:
anonimized.shape

(713, 15)

# Create result table

In [ ]:
col = ['ФИО', 'Название команды', 'Количество участников', 'Компетенция', 'Баллы, %', 'Баллы, ед.', 'Баллы по ключевым навыкам', 'Вид соревнования']

result = pd.DataFrame(columns=col)


def split_names(text):
    names = text.split(";")
    names = [name.strip() for name in names]
    return names


for filename in os.listdir(directory):
  if filename.endswith(".csv"):
    a = pd.read_csv(directory+filename)( . sys Y 
    
    if 'ФИО' in a.columns.to_list():
        for i in range(a.index.to_list()[-1]+1):
            row = a.iloc[i]
            fio = row['ФИО']
            komp = row['Компетенция']
            if komp == 'Квантовые технологии': typ = 2
            else: typ = 3
            points = row['Баллы, ед.']
            points2 = row['Баллы, %']
            try:
              points3 = row['Баллы по ключевым навыкам']
            except:
              points3 = np.NaN
            
            new_row = {'ФИО': fio, 'Название команды': fio, 'Количество участников': 1,'Компетенция': komp,'Баллы, ед.': points, 'Баллы по ключевым навыкам': points3, 'Баллы, %': points2, 'Вид соревнования': typ}
            result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)

    elif 'ФИО участников' in a.columns.to_list():
        
        for i in range(a.index.to_list()[-1]+1):
            
            row = a.iloc[i]
            team = split_names(row['ФИО участников'])
            memb_count = len(team)
            team_name = row['Название команды']
            komp = row['Компетенция']
            typ = 1
            points = row['Баллы, ед.']
            points2 = row['Баллы, %']
            points3 = row['Баллы по ключевым навыкам']
            
            for name in team:
              if komp == 'Инженерное проектирование':
                points = row['Unnamed: 9']

              new_row = {'ФИО': name, 'Название команды': team_name, 'Количество участников': memb_count,'Компетенция': komp,'Баллы, ед.': points, 'Баллы по ключевым навыкам': points3, 'Баллы, %': points2, 'Вид соревнования': typ}
              result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)

    else:
        print(filename, "Ошибка")


result['Баллы, %'] = result['Баллы, %'].fillna(result['Баллы, ед.'])

In [ ]:
result = result.drop(['Баллы, ед.', 'Баллы по ключевым навыкам'], axis=1)

result = result.rename(columns={'Баллы, %': 'Баллы'})

result = result.drop(result[result['ФИО'] == 'XXX'].index)

In [ ]:
result

,ФИО,Название команды,Количество участников,Компетенция,Баллы,Вид соревнования
0,ФИО_623,AKKUYU NÜKLEER,2,Сетевое и системное администрирование,"22,95",1
1,ФИО_703,AKKUYU NÜKLEER,2,Сетевое и системное администрирование,"22,95",1
2,ФИО_164,"АО ""АЭС"" АО ""Консист-ОС""",2,Сетевое и системное администрирование,"61,26",1
3,ФИО_310,"АО ""АЭС"" АО ""Консист-ОС""",2,Сетевое и системное администрирование,"61,26",1
4,ФИО_113,"АО ""КОНСИСТ-ОС""",2,Сетевое и системное администрирование,"59,78",1
...,...,...,...,...,...,...
707,ФИО_709,"ФГУП ""ПСЗ"" - 1",2,Управление качеством,"53,65",1
708,ФИО_382,"ФГУП ""ПСЗ"" - 2",2,Управление качеством,"44,8",1
709,ФИО_520,"ФГУП ""ПСЗ"" - 2",2,Управление качеством,"44,8",1
710,ФИО_202,Экологические решения,2,Управление качеством,"33,62",1


In [ ]:
result.to_csv('result.csv', index=False)

# Data Engineering

In [ ]:
data = pd.merge(result, anonimized, on='ФИО')

In [ ]:
data.shape

(710, 20)

In [ ]:
data = data.drop('Список компетенций', axis=1)
data = data[data['Роль в мероприятии'] == 'Участник мероприятия']
data = data.rename(columns={'Год оканчания': 'Год окончания'})

In [ ]:
data.isna().sum()

ФИО                                         0
Название команды                            0
Количество участников                       0
Компетенция                                 0
Баллы                                       0
Вид соревнования                            0
Пол                                         0
Роль в мероприятии                          0
Должность                                  72
Категория                                 484
Начало трудового стажа                    141
Место работы                                4
Профессия                                 515
Дата рождения                             192
Начало трудовой деятельности в РОСАТОМ    308
Образование                               552
Место образования                         552
Год окончания                             552
Специальность                             552
dtype: int64

In [ ]:
def calculate_age(birth_date):
    if pd.isnull(birth_date):  # Если значение ячейки пустое, оставляем его без изменений
        return birth_date
    
    current_date = datetime.now().date()
    
    # Поддерживаемые форматы даты
    date_formats = [
        "%d.%m.%Y",
        "%d.%m.%y",
        "%Y-%m-%d",
        "-%m-%d",
        "%m-%d-%Y"
    ]
    
    for date_format in date_formats:
        try:
            birth_date = datetime.strptime(birth_date, date_format).date()
            break  # Если формат успешно найден, выходим из цикла
        except ValueError:
            pass
    
    # Если формат даты не найден, возвращаем исходное значение
    if isinstance(birth_date, str):
        return birth_date
    
    age = current_date.year - birth_date.year
    # Проверяем, был ли уже день рождения в текущем году
    if (birth_date.month, birth_date.day) > (current_date.month, current_date.day):
        age -= 1
    
    return age

data['Дата рождения'] = data['Дата рождения'].apply(calculate_age)

In [ ]:
data.to_csv('data.csv', index=False)

In [ ]:
data.head()

,ФИО,Название команды,Количество участников,Компетенция,Баллы,Вид соревнования,Пол,Роль в мероприятии,Должность,Категория,Начало трудового стажа,Место работы,Профессия,Дата рождения,Начало трудовой деятельности в РОСАТОМ,Образование,Место образования,Год окончания,Специальность
0,ФИО_623,AKKUYU NÜKLEER,2,Сетевое и системное администрирование,"22,95",1,0,Участник мероприятия,Начальник участка,NaN,03.2013,AKKUYU NÜKLEER ANONİM ŞİRKETİ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ФИО_703,AKKUYU NÜKLEER,2,Сетевое и системное администрирование,"22,95",1,0,Участник мероприятия,Эксперт по технической поддержке,NaN,07.2007,АО АККУЮ НУКЛЕАР,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ФИО_164,"АО ""АЭС"" АО ""Консист-ОС""",2,Сетевое и системное администрирование,"61,26",1,0,Участник мероприятия,главный эксперт,NaN,2005-01-01,"АО ""АтомЭнергоСбыт""",NaN,36,2021-07-18,NaN,NaN,NaN,NaN
3,ФИО_310,"АО ""АЭС"" АО ""Консист-ОС""",2,Сетевое и системное администрирование,"61,26",1,1,Участник мероприятия,инженер-системотехник,NaN,май.07,"Филиал АО ""КОНСИСТ-ОС"" ""Кольский""",NaN,36,NaN,NaN,NaN,NaN,NaN
4,ФИО_113,"АО ""КОНСИСТ-ОС""",2,Сетевое и системное администрирование,"59,78",1,0,Участник мероприятия,Главный эксперт,Специалист (рабочие специальности),2007-02-01,"АО ""Консист-ОС""",Сетевое и системное администрирование,36,2007-02-01,NaN,NaN,NaN,NaN
